# Meeting June 19

## Rewrite Several Linear Algebra Operators to get rid of conversion using sparse()

In [12]:
using SparseArrays
using LinearAlgebra
using BenchmarkTools
using IterativeSolvers


# Initializing Functions
function e(i,n)
    A = Matrix{Float64}(I,n,n)
    return A[:,i]
end

function se(i,n)    # Sparse formulation of e(i,n)
    A = spzeros(n)
    A[i] = 1
    return A
end

function eyes(n)
    return Matrix{Float64}(I,n,n)
end

function speyes(n)    # Sparse formulation of eyes(n)
    A = spzeros(n,n)
    for i in 1:n
        A[i,i] = 1
    end
    return A
end

function u(x,y)
           return sin.(π*x .+ π*y)
       end

function Diag(A)
    # Self defined function that is similar to Matlab Diag
    return Diagonal(A[:])
end

function sparse_E(i,N)    ## This is because Diag was used to convert an e(i,n) to a diagonal matrix 
    A = spzeros(N,N)
    A[i,i] = 1
    return A
end

sparse_E (generic function with 1 method)

## Fix issues using @benchmark on impure function cg!()

In [3]:
function test_cg!(A_d,b_d)
    init_guess = similar(b_d)
    return cg!(init_guess,A_d,b_d)
end

function test_cg!_Pl(A_d,b_d)
    init_guess = similar(b_d)
    return cg!(init_guess,A_d,b_d;Pl=Identity())
end


function test_cg!_null(A_d,b_d)  # This function tests how long it takes to generate a random guess
    init_guess = similar(b_d)
end

test_cg!_null (generic function with 1 method)

In [24]:
A = randn(3)
B = randn(3)

3-element Array{Float64,1}:
 -0.756691453727197
  0.347406086462271
 -1.2513647684173266

In [25]:
C = randn(3)

3-element Array{Float64,1}:
 -1.298728251851834
 -0.6580543035344195
 -1.2674962528841325

In [26]:
@benchmark C = A + B

BenchmarkTools.Trial: 
  memory estimate:  112 bytes
  allocs estimate:  1
  --------------
  minimum time:     46.768 ns (0.00% GC)
  median time:      47.778 ns (0.00% GC)
  mean time:        52.460 ns (5.45% GC)
  maximum time:     1.585 μs (91.83% GC)
  --------------
  samples:          10000
  evals/sample:     990

In [28]:
C  # C is not altered after using @benchmark macro

3-element Array{Float64,1}:
 -1.298728251851834
 -0.6580543035344195
 -1.2674962528841325

In [18]:
A = randn(5,5)
A = A*A' + 2*Matrix(I,5,5)
b = randn(5)

5-element Array{Float64,1}:
  1.130360411264038
  1.318734725343321
 -1.3103624237437237
  0.12599129162501288
  1.95685472782009

In [19]:
init_guess = randn(5)

5-element Array{Float64,1}:
  0.46922760035170513
 -0.776679122517493
  0.07641386188389362
  0.5919766450954875
  1.1220360238526592

In [20]:
@benchmark cg!(init_guess,A,b)

BenchmarkTools.Trial: 
  memory estimate:  1.16 KiB
  allocs estimate:  11
  --------------
  minimum time:     580.337 ns (0.00% GC)
  median time:      604.500 ns (0.00% GC)
  mean time:        661.492 ns (6.68% GC)
  maximum time:     13.359 μs (94.43% GC)
  --------------
  samples:          10000
  evals/sample:     178

In [21]:
init_guess # But init_guess is modified during the use of @benchmark, the change is preserved after @benchmark

5-element Array{Float64,1}:
  0.4207964262565283
  0.1789937575546813
 -0.14484120345582877
  0.18020389552789762
  0.35527054888349335

## More valid results after modifications

|     | <th colspan=5>Memory (KiB)      |           |               |               |                           |                |                                    |
|-----|-------------------|-----------|---------------|---------------|---------------------------|----------------|------------------------------------|
| N   | CPU_Factorization | Solve_CPU | Iterative_GPU | Iterative_CPU | ratio (Factorization/GPU) | ratio(CPU/GPU) | ratio(Factorization/Iterative_CPU) |
| 2^3 | 154.57            | 5.27      | 5.56          | 3.66          | 27.80                     | 0.66           | 42.23                              |
| 2^4 | 434.44            | 18.45     | 5.58          | 10.53         | 77.86                     | 1.89           | 41.26                              |
| 2^5 | 1,530.00          | 68.45     | 5.89          | 35.53         | 259.76                    | 6.03           | 43.06                              |
| 2^6 | 5,650.00          | 264.36    | 5.89          | 133.34        | 959.25                    | 22.64          | 42.37                              |
| 2^7 | 21,610.00         | 1,020.00  | 5.89          | 521.34        | 3,668.93                  | 88.51          | 41.45                              |
| 2^8 | 98,750.00         | 4,030.00  | 5.89          | 2,020.00      | 16,765.70                 | 342.95         | 48.89                              |
| 2^9 | 457,640.00        | 16,060.00 | 5.89          | 8,030.00      | 77,697.79                 | 1,363.33       | 56.99                              |

|     |<th colspan=5> Time (ms)         |           |               |               |                          |                 |                         |
|-----|-------------------|-----------|---------------|---------------|--------------------------|-----------------|-------------------------|
| N   | CPU_Factorization | Solve_CPU | Iterative_GPU | Iterative_CPU | ratio (Direct_Solve/GPU) | ratio(CPU/GPU)  | ratio(Direct_Solve/CPU) |
| 2^3 | 0.15              | 0.01      | 5.70          | 0.01          | 0.001035087719           | 0.002150877193  | 0.4812398042            |
| 2^4 | 0.53              | 0.02      | 12.29         | 0.08          | 0.001920729226           | 0.006323756816  | 0.3037323037            |
| 2^5 | 2.88              | 0.11      | 22.74         | 0.53          | 0.004838398944           | 0.02327248735   | 0.2079020979            |
| 2^6 | 13.54             | 0.51      | 45.43         | <span style="color:red">0.03 </span>        | 0.0113148568             | 0.0007374248795 | <span style="color:red">15.3437416</span>              |
| 2^7 | 48.22             | 2.76      | 95.99         | 46.49         | 0.02878334879            | 0.484274895     | 0.05943597135           |
| 2^8 | 220.43            | 15.68     | 254.85        | 409.49        | 0.0615227056             | 1.606802459     | 0.0382889043            |
| 2^9 | 1,277.00          | 77.12     | 989.75        | 3,851.00      | 0.07791664562            | 3.890881536     | 0.02002544794           |

The red data are anomalies. Still the same when running this code for a second tim

## Errors vs Iteration Steps
### Maxiter = Dimension of Matrix, Divide Maxiter by 100

![Error 289](graphics/error_81.png) </br>
A: 81 by 81

![Error 289](graphics/error_289.png) </br>
A: 289 by 289

![Error 4225](graphics/error_4225.png) </br>
A: 4225 by 4225

![Error 16641](graphics/error_16641.png) </br>
A: 16641 by 16641

![Error 4225](graphics/error_66049.png) </br>
A: 66049 by 66049

![Error 4225](graphics/error_263169.png) </br>
A: 263169 by 263169

As the dimension increases, the ratio of $\frac{steps\ to\ reach\ convergence}{dimension\ of\ A}$ is decreasing

# Other explorations

### Latexify

This package [Latexify](https://korsbo.github.io/Latexify.jl/stable/) can generate sample Julia code (equations and texts) to latex format

```
ode = @ode_def PositiveFeedback begin
    dx = y*y*y / (k_y_x + y) - x - x
    dy = x^n_x / (k_x^n_x + x^n_x) - y
    end k_y k_x n_x k_y_x
    
printout = latexify(ode) |> print

$\begin{align}
\frac{dx(t)}{dt} =& \frac{\mathrm{y}\left( t \right) \mathrm{y}\left( t \right) \mathrm{y}\left( t \right)}{k_{y\_x} + \mathrm{y}\left( t \right)} - \mathrm{x}\left( t \right) - \mathrm{x}\left( t \right) \\
\frac{dy(t)}{dt} =& \frac{\left( \mathrm{x}\left( t \right) \right)^{n_{x}}}{k_{x}^{n_{x}} + \left( \mathrm{x}\left( t \right) \right)^{n_{x}}} - \mathrm{y}\left( t \right)
\end{align}
$
 
 
io = open("myfile.txt","w")
write(io,latexify(ode)) # This will save the latex code to a txt file for later use
 
```

In [41]:
using Latexify
using ParameterizedFunctions
using DiffEqBiological

In [42]:
ode = @ode_def PositiveFeedback begin
    dx = y*y*y / (k_y_x + y) - x - x
    dy = x^n_x / (k_x^n_x + x^n_x) - y
    end k_y k_x n_x k_y_x

(::PositiveFeedback{var"###ParameterizedDiffEqFunction#292",var"###ParameterizedTGradFunction#293",var"###ParameterizedJacobianFunction#294",Nothing,Nothing,ModelingToolkit.ODESystem}) (generic function with 1 method)

In [44]:
show = latexify(ode) |> display

L"$\begin{align}
\frac{dx(t)}{dt} =& \frac{\mathrm{y}\left( t \right) \mathrm{y}\left( t \right) \mathrm{y}\left( t \right)}{k_{y\_x} + \mathrm{y}\left( t \right)} - \mathrm{x}\left( t \right) - \mathrm{x}\left( t \right) \\
\frac{dy(t)}{dt} =& \frac{\left( \mathrm{x}\left( t \right) \right)^{n_{x}}}{k_{x}^{n_{x}} + \left( \mathrm{x}\left( t \right) \right)^{n_{x}}} - \mathrm{y}\left( t \right)
\end{align}
$"

In [43]:
printout = latexify(ode) |> print

$\begin{align}
\frac{dx(t)}{dt} =& \frac{\mathrm{y}\left( t \right) \mathrm{y}\left( t \right) \mathrm{y}\left( t \right)}{k_{y\_x} + \mathrm{y}\left( t \right)} - \mathrm{x}\left( t \right) - \mathrm{x}\left( t \right) \\
\frac{dy(t)}{dt} =& \frac{\left( \mathrm{x}\left( t \right) \right)^{n_{x}}}{k_{x}^{n_{x}} + \left( \mathrm{x}\left( t \right) \right)^{n_{x}}} - \mathrm{y}\left( t \right)
\end{align}
$

### DataFrame

This package [DataFrame](https://github.com/JuliaData/DataFrames.jl/blob/master/docs/src/index.md) is similar to DataFrame packages in Python, it offers many useful functions to use Julia for many kinds of data operations in Data Science 